In [1]:
import dotenv, os
import openai
from openai import OpenAI
import json
dotenv.load_dotenv()
openai.api_key  = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [2]:
result = client.batches.list()
result.data

[Batch(id='batch_k0kFYGIADP2J5F8ued9EfOzt', completion_window='24h', created_at=1725624677, endpoint='/v1/chat/completions', input_file_id='file-CdHsRgzHfTiEhZe034BFnOyy', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1725625614, error_file_id=None, errors=None, expired_at=None, expires_at=1725711077, failed_at=None, finalizing_at=1725625611, in_progress_at=1725624678, metadata={'description': 'nightly eval jobfile-CdHsRgzHfTiEhZe034BFnOyy'}, output_file_id='file-lre2fZ7MWGjYgzilDL4yZNPz', request_counts=BatchRequestCounts(completed=28, failed=0, total=28)),
 Batch(id='batch_kdwLHYNDWaVf4Rw4KDhzPhSC', completion_window='24h', created_at=1725624677, endpoint='/v1/chat/completions', input_file_id='file-t2EzdNyZuUmvoJLwfSTRjuRJ', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1725626034, error_file_id=None, errors=None, expired_at=None, expires_at=1725711077, failed_at=None, finalizing_at=1725626019, in_pro

In [3]:

# Load batch job id from a json file
jsonl_file_path = "./0906_request.jsonl"
with open(f"./{jsonl_file_path}batch_job_ids.json", "r") as f:
    batch_job_ids = json.load(f)
batch_job_ids

['batch_KoBg3DZn8Ibsx9KQ5UgDbm7J',
 'batch_lqS68bfqTjGCI0erFaB90gM3',
 'batch_kdwLHYNDWaVf4Rw4KDhzPhSC',
 'batch_k0kFYGIADP2J5F8ued9EfOzt']

In [4]:
file_responses = []
for batch_job_id in batch_job_ids:
    result = client.batches.retrieve(batch_job_id)
    file_response = client.files.content(result.output_file_id)
    file_responses += (file_response.text.split("\n"))

In [5]:
jsonl_file_responses = [json.loads(file_response) for file_response in file_responses if file_response != ""]

# save to json, responses in the response_list
with open(f"./{jsonl_file_path}_output.json", "w") as f:
    json.dump(jsonl_file_responses, f)

In [6]:
jsonl_file_responses[0]

{'id': 'batch_req_Fcn2zZt4JNVUFArjuOxa8C8z',
 'custom_id': '9762983e7ad7043d5907acecd57cc2a6b2838af12c80794d360c983c5a68e130_10621958',
 'response': {'status_code': 200,
  'request_id': '91b45f532b7572621207a7de81f8c297',
  'body': {'id': 'chatcmpl-A4S9NVuHVaCRw2vOyIiAPDtsIiPKi',
   'object': 'chat.completion',
   'created': 1725624885,
   'model': 'gpt-4o-2024-08-06',
   'choices': [{'index': 0,
     'message': {'role': 'assistant',
      'content': '<markdown>\n# An Infinite Descent into Pure Mathematics\n\n**by Clive Newstead**\n\n*Version 0.5*  \n*Last updated on Saturday 24th December 2022*  \n[https://infinitedescent.xyz](https://infinitedescent.xyz)\n</markdown>',
      'refusal': None},
     'logprobs': None,
     'finish_reason': 'stop'}],
   'usage': {'prompt_tokens': 1208,
    'completion_tokens': 63,
    'total_tokens': 1271},
   'system_fingerprint': 'fp_8e1177b306'}},
 'error': None}

In [18]:
# output them to textbook.md
with open(f"./{jsonl_file_path}_output.md", "w") as f:
    for response in jsonl_file_responses:
        f.write(response['response']['body']['choices'][0]['message']['content'])

'<markdown>\n# An Infinite Descent into Pure Mathematics\n\n**by Clive Newstead**\n\n*Version 0.5*  \n*Last updated on Saturday 24th December 2022*  \n[https://infinitedescent.xyz](https://infinitedescent.xyz)\n</markdown>'